In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from scipy.stats import f_oneway
from scipy.stats import linregress

In [ ]:
df_scPDSI = pd.read_csv('annual_scPDSI.csv')
df_pre = pd.read_csv('annual_pre.csv')
df_temp = pd.read_csv('annual_temp.csv')
df_tmax = pd.read_csv('annual_Tmax.csv')
df_tmin = pd.read_csv('annual_Tmin.csv')
df_dtr = pd.read_csv('annual_dtr.csv')
df_pet = pd.read_csv('annual_pet.csv')
df_vpd = pd.read_csv('annual_VPD.csv')
df_sample = pd.read_csv('sample_site.csv')

In [ ]:
def get_dry_wet_years(df):
    dry_years = {}
    wet_years = {}

    for site in df.index:
        dry_years[site] = []
        wet_years[site] = []

        for year in df.columns:
            value = df.loc[site, year]

            if -4 < value < -0.5:
                dry_years[site].append(year)
            elif 0.5 < value < 4:
                wet_years[site].append(year)

    return dry_years, wet_years

dry_years, wet_years = get_dry_wet_years(df_scPDSI.dropna().set_index('site'))

In [ ]:
def intersection_list_ordered(list1, list2):
    result = [value for value in list1 if value in list2]

    return result

list1 = [1, 2, 3, 4, 5]
list2 = [3, 4, 5, 6, 7]

result = intersection_list_ordered(list1, list2)
print(result)

In [ ]:
def calculate_row_means(df1,df2):
    df_concat = pd.merge(df1,df2, on = 'site')
    df_concat.loc[:,'1901':].mean(axis= 1)
    return df_concat.loc[:,'1901':].mean(axis= 1).to_list()

In [ ]:
list_Growth_loss = []
from tqdm import tqdm
site_list =pd.merge(df_sample,df_scPDSI.dropna().reset_index(),on = 'site')['site'].to_list()
for i in tqdm(range(len(site_list)), desc="Calculating Growth_loss"):
    site_i = site_list[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i =pd.read_csv(std_path)
    #Growth-no-drought
    wet_years_i  = [int(year) for year in wet_years[site_i]]
    wet_years_i = [year for year in wet_years_i if year <= 2015]
    wet_years_i_intersection = intersection_list_ordered(wet_years_i,df_rwl_i.index.to_list())
    Growth_no_dry = df_rwl_i.loc[wet_years_i_intersection,'std'].mean()
    #Growth-drought
    dry_years_i  = [int(year) for year in dry_years[site_i]]
    dry_years_i = [year for year in dry_years_i if year <= 2015]
    dry_years_i_intersection = intersection_list_ordered(dry_years_i,df_rwl_i.index.to_list())
    Growth_dry = df_rwl_i.loc[dry_years_i_intersection,'std'].mean()
    Growth_loss = (Growth_no_dry-Growth_dry)/Growth_no_dry
    list_Growth_loss.append(Growth_loss)

In [ ]:
cs = pd.merge(df_sample,df_scPDSI.dropna().reset_index()['site'],on = 'site')

In [ ]:
scpDSI_mean = calculate_row_means(cs,df_scPDSI)
pre_mean = calculate_row_means(cs,df_pre)
temp_mean = calculate_row_means(cs,df_temp)
tmax_mean = calculate_row_means(cs,df_tmax)
tmin_mean = calculate_row_means(cs,df_tmin)
dtr_mean = calculate_row_means(cs,df_dtr)
pet_mean = calculate_row_means(cs,df_pet)
vpd_mean = calculate_row_means(cs,df_vpd)

In [ ]:
mean_Groth_loss = {'site':site_list,'Growth_loss':list_Growth_loss,'pre_mean':pre_mean,
           'scpDSI_mean':scpDSI_mean,'temp_mean':temp_mean,'tmax_mean':tmax_mean,
                  'tmin_mean':tmin_mean,'dtr_mean':dtr_mean,'pet_mean':pet_mean,
                'VPD_mean': vpd_mean }
df_mean_Groth_loss = pd.DataFrame(mean_Groth_loss)
df_merge = pd.merge(df_mean_Groth_loss,df_sample,on='site')
df_merge = df_merge.dropna()
df_merge.to_csv('GrowthLoss.csv')
Arid = df_merge[df_merge['AI_Class']=='Arid']
Humid = df_merge[df_merge['AI_Class']=='Humid']

In [ ]:
def calculate_confidence_interval(df, column):
    lower_bound = np.percentile(df[column], 2.5)
    upper_bound = np.percentile(df[column], 97.5)
    df_95 = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    mean_value = np.mean(df_95[column])
    print("95% Confidence Interval ({}): [{:.4f}, {:.4f}]".format(column, lower_bound, upper_bound))
    print("Mean ({}): {:.4f}".format(column, mean_value))
    return df_95

In [ ]:
grouped = df_merge.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'Growth_loss')
group_arid95 = calculate_confidence_interval(group_arid, 'Growth_loss')
df_merge95 = pd.concat([group_humid95,group_arid95],axis=0)
f_statistic_dr, p_value_dr = f_oneway(group_humid95['Growth_loss'], group_arid95['Growth_loss'])
anova_results_dr['Growth_loss'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}

for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
df_merge95.to_csv('GLaridHumid_95.csv',index=False)    

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
colors = [(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
plt.figure(figsize=(4,3.5),dpi=100)
#palette=['#8DCDD5','#E6846D']
alpha = 0.5 
sns.boxplot(x="AI_Class", y="Growth_loss", data=df_merge95
            ,palette=colors,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.ylim(-0.3,0.6)
plt.yticks([-0.3,0,0.3,0.6]) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('Growth loss',fontsize=18)
plt.text(0.5, 0.5, '***', ha='center', va='center')
plt.text('Humid', 0.5,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.5,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('Fig2c.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_merge95['Latitude_bin'] = pd.cut(df_merge95['Latitude'], bins=np.arange(-90, 91, 1))
latitude_stats = df_merge95.groupby('Latitude_bin')['Growth_loss'].agg(['mean', 'sem']).reset_index()
slope, intercept, r_value, p_value, std_err = linregress(latitude_stats.dropna()['mean'], latitude_stats.dropna()['Latitude_bin'].apply(lambda x: x.mid))


fig, ax = plt.subplots(figsize=(3, 6))
ax.axvline(x=0, color='gray', linestyle='--')
ax.plot(latitude_stats['mean'], latitude_stats['Latitude_bin'].apply(lambda x: x.mid), marker='o', linestyle='-', color='black')
ax.fill_betweenx(latitude_stats['Latitude_bin'].apply(lambda x: x.mid), 
                 latitude_stats['mean'] - 1.96 * latitude_stats['sem'], 
                 latitude_stats['mean'] + 1.96 * latitude_stats['sem'], 
                 color='orange', alpha=0.3)


ax.plot(latitude_stats.dropna()['mean'], intercept + slope * latitude_stats.dropna()['mean'],color = 'blue')
ax.set_xlabel('Growth Loss')
ax.set_ylabel('Latitude (°)')
ax.set_xlim(-0.2, 0.3) 
ax.grid(True)
ax.text(0.05, 0.35, f'r ={r_value:.2f}\np ={p_value:.2e}', transform=ax.transAxes
        , fontsize=14, verticalalignment='top')
plt.ylim(15,75)
plt.savefig('FigS5a.jpg', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('default')
plt.rcParams["xtick.labelsize"] = 36
plt.rcParams["ytick.labelsize"] = 36
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 36
negative_percentage = (df_merge95['Growth_loss'] < 0).mean() * 100
positive_percentage = (df_merge95['Growth_loss'] > 0).mean() * 100

data = pd.DataFrame({
    'Category': ['Negative', 'Positive'],
    'Percentage': [negative_percentage, positive_percentage]
})
plt.figure(figsize=(2,3.5),dpi=100)
colors = [(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]

x = [0.3, 0.2]

fig, ax = plt.subplots()
bars = ax.bar(x, data['Percentage'], color=colors, width=0.06,alpha=0.7)

ax.set_xticks(x)
ax.set_xticklabels(data['Category'])

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.0f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, -20), textcoords="offset points", ha='center', va='bottom')  # 标注下移一点

plt.savefig('Fig2aGLbars.jpg',dpi=600)
plt.show()